# Factorizations and other fun
Based on work by Andreas Noack

## Outline
 - Factorizations
 - Special matrix structures
 - Generic linear algebra

Before we get started, let's set up a linear system and use `LinearAlgebra` to bring in the factorizations and special matrix structures.

In [1]:
using LinearAlgebra
A = rand(3, 3)
x = fill(1, (3,))
b = A * x

3-element Vector{Float64}:
 1.4818578127147206
 1.2756762015937477
 1.2252374618722235

## Factorizations

#### LU factorizations
In Julia we can perform an LU factorization
```julia
PA = LU
``` 
where `P` is a permutation matrix, `L` is lower triangular unit diagonal and `U` is upper triangular, using `lufact`.

Julia allows computing the LU factorization and defines a composite factorization type for storing it.

In [2]:
Alu = lu(A)

LU{Float64, Matrix{Float64}, Vector{Int64}}
L factor:
3×3 Matrix{Float64}:
 1.0       0.0       0.0
 0.277213  1.0       0.0
 0.301792  0.984618  1.0
U factor:
3×3 Matrix{Float64}:
 0.36145  0.146818   0.97359
 0.0      0.917951  -0.103504
 0.0      0.0        0.0265435

In [3]:
typeof(Alu)

LU{Float64, Matrix{Float64}, Vector{Int64}}

The different parts of the factorization can be extracted by accessing their special properties

In [4]:
Alu.P

3×3 Matrix{Float64}:
 1.0  0.0  0.0
 0.0  0.0  1.0
 0.0  1.0  0.0

In [5]:
Alu.L

3×3 Matrix{Float64}:
 1.0       0.0       0.0
 0.277213  1.0       0.0
 0.301792  0.984618  1.0

In [6]:
Alu.U

3×3 Matrix{Float64}:
 0.36145  0.146818   0.97359
 0.0      0.917951  -0.103504
 0.0      0.0        0.0265435

Julia can dispatch methods on factorization objects.

For example, we can solve the linear system using either the original matrix or the factorization object.

In [7]:
A\b

3-element Vector{Float64}:
 0.9999999999999803
 1.0000000000000007
 1.0000000000000073

In [8]:
Alu\b

3-element Vector{Float64}:
 0.9999999999999803
 1.0000000000000007
 1.0000000000000073

Similarly, we can calculate the determinant of `A` using either `A` or the factorization object

In [9]:
det(A) ≈ det(Alu)

true

#### QR factorizations

In Julia we can perform a QR factorization
```
A=QR
``` 

where `Q` is unitary/orthogonal and `R` is upper triangular, using `qrfact`. 

In [10]:
Aqr = qr(A)

LinearAlgebra.QRCompactWY{Float64, Matrix{Float64}, Matrix{Float64}}
Q factor:
3×3 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}, Matrix{Float64}}:
 -0.925321   0.378628   0.0205484
 -0.279255  -0.643802  -0.712415
 -0.256511  -0.664951   0.701457
R factor:
3×3 Matrix{Float64}:
 -0.390621  -0.646531  -1.00457
  0.0       -1.19228    0.117348
  0.0        0.0       -0.01891

Similarly to the LU factorization, the matrices `Q` and `R` can be extracted from the QR factorization object via

In [11]:
Aqr.Q

3×3 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}, Matrix{Float64}}:
 -0.925321   0.378628   0.0205484
 -0.279255  -0.643802  -0.712415
 -0.256511  -0.664951   0.701457

In [12]:
Aqr.R

3×3 Matrix{Float64}:
 -0.390621  -0.646531  -1.00457
  0.0       -1.19228    0.117348
  0.0        0.0       -0.01891

#### Eigendecompositions

The results from eigendecompositions, singular value decompositions, Hessenberg factorizations, and Schur decompositions are all stored in `Factorization` types.

The eigendecomposition can be computed

In [13]:
Asym = A + A'
AsymEig = eigen(Asym)

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
3-element Vector{Float64}:
 -0.7991849851968578
  0.9366495698648938
  2.814490627747772
vectors:
3×3 Matrix{Float64}:
 -0.515293   0.776436  -0.362795
 -0.301424  -0.560473  -0.771372
  0.802257   0.288128  -0.522844

The values and the vectors can be extracted from the Eigen type by special indexing

In [14]:
AsymEig.values

3-element Vector{Float64}:
 -0.7991849851968578
  0.9366495698648938
  2.814490627747772

In [15]:
AsymEig.vectors

3×3 Matrix{Float64}:
 -0.515293   0.776436  -0.362795
 -0.301424  -0.560473  -0.771372
  0.802257   0.288128  -0.522844

Once again, when the factorization is stored in a type, we can dispatch on it and write specialized methods that exploit the properties of the factorization, e.g. that $A^{-1}=(V\Lambda V^{-1})^{-1}=V\Lambda^{-1}V^{-1}$.

In [16]:
inv(AsymEig)*Asym

3×3 Matrix{Float64}:
  1.0           2.22045e-15  -3.66374e-15
  8.32667e-16   1.0          -1.45717e-15
 -2.33147e-15  -2.44249e-15   1.0

## Special matrix structures
Matrix structure is very important in linear algebra. To see *how* important it is, let's work with a larger linear system

In [17]:
n = 1000
A = randn(n,n);

Julia can often infer special matrix structure

In [18]:
Asym = A + A'
issymmetric(Asym)

true

but sometimes floating point error might get in the way.

In [19]:
Asym_noisy = copy(Asym)
Asym_noisy[1,2] += 5eps()

-0.3235256760775307

In [20]:
issymmetric(Asym_noisy)

false

Luckily we can declare structure explicitly with, for example, `Diagonal`, `Triangular`, `Symmetric`, `Hermitian`, `Tridiagonal` and `SymTridiagonal`.

In [21]:
Asym_explicit = Symmetric(Asym_noisy);

Let's compare how long it takes Julia to compute the eigenvalues of `Asym`, `Asym_noisy`, and `Asym_explicit`

In [22]:
@time eigvals(Asym);

  0.502179 seconds (293.34 k allocations: 22.382 MiB, 68.35% compilation time)


In [23]:
@time eigvals(Asym_noisy);

  1.485809 seconds (13 allocations: 7.920 MiB, 2.82% gc time)


In [24]:
@time eigvals(Asym_explicit);

  0.209240 seconds (13.69 k allocations: 8.708 MiB, 10.51% compilation time)


In this example, using `Symmetric()` on `Asym_noisy` made our calculations about `5x` more efficient :)

#### A big problem
Using the `Tridiagonal` and `SymTridiagonal` types to store tridiagonal matrices makes it possible to work with potentially very large tridiagonal problems. The following problem would not be possible to solve on a laptop if the matrix had to be stored as a (dense) `Matrix` type.

In [25]:
n = 1_000_000;
A = SymTridiagonal(randn(n), randn(n-1));
@time eigmax(A)

  1.699580 seconds (1.18 M allocations: 243.189 MiB, 19.37% gc time, 53.82% compilation time)


6.374413750315135

## Generic linear algebra
The usual way of adding support for numerical linear algebra is by wrapping BLAS and LAPACK subroutines. For matrices with elements of `Float32`, `Float64`, `Complex{Float32}` or `Complex{Float64}` this is also what Julia does.

However, Julia also supports generic linear algebra, allowing you to, for example, work with matrices and vectors of rational numbers.

#### Rational numbers
Julia has rational numbers built in. To construct a rational number, use double forward slashes:

In [26]:
1//2

1//2

#### Example: Rational linear system of equations
The following example shows how linear system of equations with rational elements can be solved without promoting to floating point element types. Overflow can easily become a problem when working with rational numbers so we use `BigInt`s.

In [27]:
Arational = Matrix{Rational{BigInt}}(rand(1:10, 3, 3))/10

3×3 Matrix{Rational{BigInt}}:
 7//10  9//10  3//10
 1//1   2//5   4//5
 9//10  1//2   9//10

In [28]:
x = fill(1, 3)
b = Arational*x

3-element Vector{Rational{BigInt}}:
 19//10
 11//5
 23//10

In [29]:
Arational\b

3-element Vector{Rational{BigInt}}:
 1//1
 1//1
 1//1

In [30]:
lu(Arational)

LU{Rational{BigInt}, Matrix{Rational{BigInt}}, Vector{Int64}}
L factor:
3×3 Matrix{Rational{BigInt}}:
 1//1   0//1   0//1
 7//10  1//1   0//1
 9//10  7//31  1//1
U factor:
3×3 Matrix{Rational{BigInt}}:
 1//1   2//5     4//5
 0//1  31//50  -13//50
 0//1   0//1    37//155

### Exercises

#### 11.1
What are the eigenvalues of matrix A?

```
A =
[
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36
]
```
and assign it a variable `A_eigv`

In [31]:
using LinearAlgebra

In [41]:
A = [140 97 74 168 131; 97 106 89 131 36; 74 89 152 144 71; 168 131 144 54 142; 131 36 71 142 36]
A_eigv = eigvals(A)

5-element Vector{Float64}:
 -128.49322764802145
  -55.887784553056875
   42.7521672793189
   87.16111477514521
  542.4677301466143

In [42]:
@assert A_eigv == [-128.49322764802145, -55.887784553056875, 42.7521672793189, 87.16111477514521, 542.4677301466143]

#### 11.2 
Create a `Diagonal` matrix from the eigenvalues of `A`.

In [76]:
A_diag = Diagonal(A_eigv)

5×5 Diagonal{Float64, Vector{Float64}}:
 -128.493     ⋅        ⋅        ⋅         ⋅ 
     ⋅     -55.8878    ⋅        ⋅         ⋅ 
     ⋅        ⋅      42.7522    ⋅         ⋅ 
     ⋅        ⋅        ⋅      87.1611     ⋅ 
     ⋅        ⋅        ⋅        ⋅      542.468

In [77]:
@assert A_diag == [-128.49322764802145 0.0 0.0 0.0 0.0;
    0.0 -55.887784553056875 0.0 0.0 0.0;
    0.0 0.0 42.7521672793189 0.0 0.0;
    0.0 0.0 0.0 87.16111477514521 0.0;
    0.0 0.0 0.0 0.0 542.4677301466143]

#### 11.3 
Create a `LowerTriangular` matrix from `A` and store it in `A_lowertri`

In [78]:
A_lowertri = LowerTriangular(A)

5×5 LowerTriangular{Int64, Matrix{Int64}}:
 140    ⋅    ⋅    ⋅   ⋅
  97  106    ⋅    ⋅   ⋅
  74   89  152    ⋅   ⋅
 168  131  144   54   ⋅
 131   36   71  142  36

In [75]:
@assert A_lowertri ==  [140    0    0    0   0;
  97  106    0    0   0;
  74   89  152    0   0;
 168  131  144   54   0;
 131   36   71  142  36]